In [48]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns

In [49]:
df = pd.read_csv('data/cleaned_video_game.csv')

In [51]:
print(df.shape)
df

(9376, 29)


,Name,Console,Year_of_Release,ESRB_rating,Publisher,Global_players,Genre,Critic_Score,User_Score,NA_Sales,...,Accessory,LtdEdition,Multiplatform,RealTime,TurnBased,VisualNovel,FirstPerson,SideScrolling,TopDown,ThirdPerson
0,Tales of Xillia 2,PS3,2012,T,Namco Bandai Games,0.84,Role-Playing,71.0,7.9,0.20,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,.hack//Infection Part 1,PS2,2002,T,Atari,1.27,Role-Playing,75.0,8.5,0.49,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,.hack//Mutation Part 2,PS2,2002,T,Atari,0.68,Role-Playing,76.0,8.9,0.23,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,.hack//Outbreak Part 3,PS2,2002,T,Atari,0.46,Role-Playing,70.0,8.7,0.14,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,007 Racing,PS,2000,T,Electronic Arts,0.53,Racing,51.0,4.6,0.30,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,007: Quantum of Solace,DS,2008,T,Activision,0.13,Action,65.0,8.7,0.11,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,007: Quantum of Solace,PC,2008,T,Activision,0.03,Action,70.0,6.3,0.01,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,007: Quantum of Solace,PS3,2008,T,Activision,1.14,Action,65.0,6.6,0.43,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,007: Quantum of Solace,Wii,2008,T,Activision,0.65,Action,54.0,7.5,0.29,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,007: Quantum of Solace,X360,2008,T,Activision,1.48,Action,65.0,7.1,0.82,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [53]:
df.ESRB_rating.value_counts()

E       3656
T       2828
M       1533
E10+    1347
EC         8
K-A        2
AO         1
RP         1
Name: ESRB_rating, dtype: int64